In [53]:
import sklearn
import numpy as np
import pandas as pd
import os
import time
from sklearn.preprocessing import OneHotEncoder

https://data.cityofnewyork.us/Housing-Development/NYC-Calendar-Sales-Archive-/uzf5-f8n2  <br>
https://data.cityofnewyork.us/City-Government/NYC-Citywide-Annualized-Calendar-Sales-Update/w2pb-icbu

## Combine all of the CSV files into one dataframe

In [2]:
folder_name = 'data'
data_dir = os.path.join(os.getcwd(),folder_name)
all_sales_df = pd.DataFrame() 

for file_name in os.listdir(data_dir):

    if file_name[0:4] in ['2011','2012','2013','2014','2015']:
        temp_frame = pd.read_excel(os.path.join(data_dir,file_name),header = 4)
    elif file_name[0:3] == 'NYC':
        temp_frame = pd.read_csv(os.path.join(data_dir,file_name),header = 0)
        temp_frame = temp_frame.rename(columns = {'TAX CLASS AS OF FINAL ROLL':'TAX CLASS AT PRESENT','BUILDING CLASS AS OF FINAL ROLL':'BUILDING CLASS AT PRESENT'})
        # temp_frame = temp_frame.rename(columns = {'EASE-MENT':'EASEMENT'})
        #Dropping columns that only appear in most recent data
        temp_frame = temp_frame.drop(columns = ["Latitude","Longitude","Community Board","Council District","Census Tract","BIN","BBL","NTA"])
    else:
        temp_frame = pd.read_excel(os.path.join(data_dir,file_name),header = 3)
    temp_frame.columns = temp_frame.columns.str.rstrip()
    if all_sales_df.empty == True:
        all_sales_df = temp_frame
    else:
        all_sales_df = all_sales_df.append(temp_frame)
#Dropping EASEMENT as there are only 13 in half a million records
#Dropping columns that aren't useful
#Using Apartment number would be nice to determine unit location in building if we have time
all_sales_df = all_sales_df.drop(columns = ["NEIGHBORHOOD","ADDRESS","TAX CLASS AT PRESENT","BUILDING CLASS AT PRESENT","LOT","EASE-MENT","APARTMENT NUMBER"])
#I need to reference these later so it cant have a space
all_sales_df = all_sales_df.rename(columns = {"SALE PRICE":"SALE_PRICE","RESIDENTIAL UNITS":"RESIDENTIAL_UNITS","LAND SQUARE FEET":"LAND_SQUARE_FEET","GROSS SQUARE FEET":"GROSS_SQUARE_FEET"})
int_cols = ['RESIDENTIAL_UNITS','COMMERCIAL UNITS','TOTAL UNITS','LAND SQUARE FEET','GROSS SQUARE FEET','YEAR BUILT','SALE_PRICE']
all_sales_df.replace(',','',regex=True,inplace = True)

C:\Users\Will\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
all_sales_df.head()

,BOROUGH,BUILDING CLASS CATEGORY,BLOCK,ZIP CODE,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE_PRICE,SALE DATE
0,2,01 ONE FAMILY HOMES,3035,10457.0,1,0,1,1287,1992,1899,1,A9,0,2009-12-07 00:00:00
1,2,01 ONE FAMILY HOMES,3046,10457.0,1,0,1,1964,1424,1899,1,A1,100,2009-04-28 00:00:00
2,2,01 ONE FAMILY HOMES,3046,10457.0,1,0,1,1964,1424,1899,1,A1,100,2009-04-13 00:00:00
3,2,01 ONE FAMILY HOMES,3048,10457.0,1,0,1,1768,1188,1901,1,A5,345752,2009-03-16 00:00:00
4,2,01 ONE FAMILY HOMES,3048,10457.0,1,0,1,1209,1048,1901,1,A1,0,2009-05-14 00:00:00


In [4]:
all_sales_df.columns

Index(['BOROUGH', 'BUILDING CLASS CATEGORY', 'BLOCK', 'ZIP CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE_PRICE', 'SALE DATE'],
      dtype='object')

In [5]:
for col in all_sales_df.columns:
    print(col, len(all_sales_df[col].unique()))

BOROUGH 5
BUILDING CLASS CATEGORY 145
BLOCK 13617
ZIP CODE 242
RESIDENTIAL_UNITS 740
COMMERCIAL UNITS 262
TOTAL UNITS 796
LAND_SQUARE_FEET 29023
GROSS_SQUARE_FEET 32030
YEAR BUILT 391
TAX CLASS AT TIME OF SALE 4
BUILDING CLASS AT TIME OF SALE 215
SALE_PRICE 106914
SALE DATE 6207


## DATA CLEANING

In [6]:
#Removing incomplete records, almost all of the dropped records had no sales price
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE != 0]
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE != -0]
all_sales_df = all_sales_df[all_sales_df.LAND_SQUARE_FEET != '-0']
all_sales_df = all_sales_df[all_sales_df.GROSS_SQUARE_FEET != '-0']
all_sales_df = all_sales_df[all_sales_df.LAND_SQUARE_FEET != '- 0']
all_sales_df = all_sales_df[all_sales_df.GROSS_SQUARE_FEET != '- 0']



all_sales_df = all_sales_df.dropna()


In [7]:
all_sales_df.shape

(1115050, 14)

In [8]:
all_sales_df = all_sales_df.astype({'RESIDENTIAL_UNITS':'int64'})
all_sales_df = all_sales_df.astype({'COMMERCIAL UNITS':'int64'})
all_sales_df = all_sales_df.astype({'TOTAL UNITS':'int64'})
all_sales_df = all_sales_df.astype({'LAND_SQUARE_FEET':'int64'})
all_sales_df = all_sales_df.astype({'GROSS_SQUARE_FEET':'int64'})
all_sales_df = all_sales_df.astype({'YEAR BUILT':'int64'})
all_sales_df = all_sales_df.astype({'SALE_PRICE':'int64'})


In [9]:
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE <= 10000000]
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE >= 100000]

In [10]:
all_sales_df.shape

(932459, 14)

In [11]:
all_sales_df.head()

,BOROUGH,BUILDING CLASS CATEGORY,BLOCK,ZIP CODE,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE_PRICE,SALE DATE
3,2,01 ONE FAMILY HOMES,3048,10457.0,1,0,1,1768,1188,1901,1,A5,345752,2009-03-16 00:00:00
6,2,02 TWO FAMILY HOMES,3030,10457.0,2,0,2,2352,1632,1899,1,B2,135000,2009-05-08 00:00:00
7,2,02 TWO FAMILY HOMES,3030,10457.0,2,0,2,2444,2125,1899,1,B2,325000,2009-03-30 00:00:00
10,2,02 TWO FAMILY HOMES,3036,10457.0,2,0,2,2000,2340,1998,1,B1,344500,2009-11-20 00:00:00
11,2,02 TWO FAMILY HOMES,3036,10457.0,2,0,2,2000,2340,1998,1,B1,316474,2009-05-04 00:00:00


In [12]:
all_sales_df['SALE DATE'] = pd.to_datetime(all_sales_df['SALE DATE'])

In [18]:
all_sales_df.to_csv('Cleaned_Sales_Data.csv')

## HELPER FUNCTIONS

In [13]:
def mean_percent_error(yTrue,yPred, minPrice=0, maxPrice=999999999999999):
    count = 0
    percent_error = 0.0
    for i in range(len(yTrue)):    
        if yTrue[i] >= minPrice and yTrue[i] <= maxPrice:
            count +=1
            percent_error += (abs(yPred[i]/yTrue[i])-1.0)
    return percent_error/float(count)

In [14]:
def mae_by_price_band(yTrue,yPred, minPrice=0, maxPrice=999999999999999):
    count = 0
    error = 0.0
    for i in range(len(yTrue)):
        if yTrue[i] >= minPrice and yTrue[i] <= maxPrice:
            count += 1
            error += abs(yTrue[i]-yPred[i])
    return error / float(count)
    

In [15]:
# Find the polynomial feature excluding combination of one hot vectors to keep feature count down

#This made things worse.  Curse of Dimensionallity with the one hots?
def PolyFeat(source_df, one_hot_list):
    non_oh_cols = []
    oh_cols = []
    for col in source_df.columns:
        oh_col = False
        for oh in one_hot_list:
            if col.startswith(oh):
                oh_col = True
        if oh_col == True:
            oh_cols.append(col)
        else:
            non_oh_cols.append(col)
    for non_oh_col in non_oh_cols:
        for one_hot_col in oh_cols:
            new_col = non_oh_col + '_' + one_hot_col
            source_df[new_col] = source_df[non_oh_col]*source_df[one_hot_col]
    return source_df
                


In [16]:
def month_distance(start, end):
    mdiff = (end.year - start.year) * 12 + end.month - start.month
    return mdiff

In [17]:
# Take the most recent date in df, count backwards by month for time series info
def Convert_dates(source_df, index_name = 'SALE DATE', new_col_name = 'Date'):
    source_df.reset_index(inplace=True)
    source_df[new_col_name] = 0
    max_date = source_df[index_name].max()
    for i, row in source_df.iterrows():
        source_df.at[i,new_col_name] = month_distance(row[index_name],max_date)
    source_df.set_index(index_name, inplace=True)
    source_df = pd.get_dummies(source_df, columns = [new_col_name], prefix = new_col_name)
    return source_df
    

## PREPARING SMALLER TRAIN AND TEST DATASETS

In [54]:
scikit_df = all_sales_df.copy()
# one_hots = ['BOROUGH','NEIGHBORHOOD','BUILDING CLASS CATEGORY','ZIP CODE','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE']
one_hots = ['BOROUGH','BUILDING CLASS CATEGORY','ZIP CODE','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE']
scikit_df = pd.get_dummies(scikit_df, columns = one_hots, prefix = one_hots)
scikit_df.drop('BLOCK',inplace = True, axis = 1)

In [55]:
# scikit_df.to_csv('processed.csv')

In [56]:
scikit_df.shape

(932459, 574)

In [57]:
scikit_df.set_index('SALE DATE', inplace=True)

In [58]:
scikit_df.head()

,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,SALE_PRICE,BOROUGH_1,BOROUGH_2,BOROUGH_3,...,BUILDING CLASS AT TIME OF SALE_Z0,BUILDING CLASS AT TIME OF SALE_Z1,BUILDING CLASS AT TIME OF SALE_Z2,BUILDING CLASS AT TIME OF SALE_Z3,BUILDING CLASS AT TIME OF SALE_Z4,BUILDING CLASS AT TIME OF SALE_Z5,BUILDING CLASS AT TIME OF SALE_Z6,BUILDING CLASS AT TIME OF SALE_Z7,BUILDING CLASS AT TIME OF SALE_Z8,BUILDING CLASS AT TIME OF SALE_Z9
SALE DATE,,,,,,,,,,,,,,,,,,,,,
2009-03-16,1,0,1,1768,1188,1901,345752,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2009-05-08,2,0,2,2352,1632,1899,135000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2009-03-30,2,0,2,2444,2125,1899,325000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2009-11-20,2,0,2,2000,2340,1998,344500,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2009-05-04,2,0,2,2000,2340,1998,316474,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
date_range_sample = ['2005-01-01' , '2007-12-31']
date_range_target = ['2008-01-01' , '2008-01-31']

In [60]:
#Remove any columns in the data range that don't have any hits on the one hot
temp_df = scikit_df.loc[date_range_sample[0] : date_range_sample[1]]
print(temp_df.shape)
scikit_dr_df = temp_df.loc[:, (temp_df != 0).any(axis=0)]
print(scikit_dr_df.shape)

#Take the same columns as the sample date range so target date range can be passed to scikit
temp_target_df = scikit_df.loc[date_range_target[0] : date_range_target[1]]
scikit_dr_Y_df = temp_target_df[scikit_dr_df.columns]

(229476, 573)
(229476, 394)


In [61]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [62]:
scikit_dr_df = Convert_dates(scikit_dr_df)
scikit_dr_df

<ipython-input-17-caac207595d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df[new_col_name] = 0


,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,SALE_PRICE,BOROUGH_1,BOROUGH_2,BOROUGH_3,...,Date_26,Date_27,Date_28,Date_29,Date_30,Date_31,Date_32,Date_33,Date_34,Date_35
SALE DATE,,,,,,,,,,,,,,,,,,,,,
2007-02-16,1,0,1,1165,1488,1899,475000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2007-06-15,1,0,1,1782,1548,1899,376300,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2007-05-23,1,0,1,3525,1340,1899,200000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2007-09-12,0,0,0,0,0,0,950000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2007-01-11,1,0,1,1556,1240,1901,172000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-03-27,2,0,2,5009,2796,2001,730000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-08-09,2,0,2,4000,2234,2006,712775,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-06-28,2,0,2,5500,2688,2007,365000,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
for col in scikit_dr_df:
    if col.startswith('Date_'):
        scikit_dr_Y_df[col] = 0
# scikit_dr_Y_df = Convert_dates(scikit_dr_Y_df)
scikit_dr_Y_df

<ipython-input-63-02a495c5d39d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scikit_dr_Y_df[col] = 0


,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,SALE_PRICE,BOROUGH_1,BOROUGH_2,BOROUGH_3,...,Date_26,Date_27,Date_28,Date_29,Date_30,Date_31,Date_32,Date_33,Date_34,Date_35
SALE DATE,,,,,,,,,,,,,,,,,,,,,
2008-01-18,2,0,2,1556,1494,1899,510000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-31,1,0,1,1710,1890,1945,350000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-17,1,0,1,2160,1560,1950,350000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-16,1,0,1,1900,2940,1960,350000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-18,1,0,1,2300,1620,1950,395000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-01-17,1,0,1,2852,2307,2001,430000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-25,2,0,2,3700,2704,1996,493000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-29,2,0,2,3215,1900,1995,470000,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
x_train = scikit_dr_df.drop(columns =['SALE_PRICE'],axis = 1)
y_train = scikit_dr_df['SALE_PRICE']

x_test = scikit_dr_Y_df.drop(columns =['SALE_PRICE'],axis = 1)
y_test = scikit_dr_Y_df['SALE_PRICE']


## Regression experiments

In [30]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

### Train

In [31]:
reg = LinearRegression().fit(x_train,y_train)
preds = reg.predict(x_train)
print('MAE',mean_absolute_error(scikit_dr_df['SALE_PRICE'],preds))
print('MPE',mean_percent_error(y_train.tolist(), preds))
print('MAE Band',mae_by_price_band(y_train.tolist(), preds,650000,750000))


MAE 328455.2782543426
MPE 0.30153126666975993
MAE Band 227116.13475486488


### Test

In [32]:
preds = reg.predict(x_test)
print('MAE',mean_absolute_error(y_test, preds))
print('MPE',mean_percent_error(y_test.tolist(), preds))
print('MAE Band',mae_by_price_band(y_test.tolist(), preds,500000,650000))

MAE 417805.1833239149
MPE 0.2684458547505265
MAE Band 229082.07986782942


## RIDGE

In [40]:
ridge = Ridge().fit(x_train,y_train)
preds = ridge.predict(x_train)
print('MAE',mean_absolute_error(y_train,preds))
print('MPE',mean_percent_error(y_train.tolist(), preds))
print('MAE Band',mae_by_price_band(y_train.tolist(), preds,650000,750000))

C:\Users\Will\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.86124e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


MAE 328550.48110523913
MPE 0.30199134134612443
MAE Band 227126.24123329337


In [41]:
preds = ridge.predict(x_test)
print('MAE',mean_absolute_error(y_test, preds))
print('MPE',mean_percent_error(y_test.tolist(), preds))
print('MAE Band',mae_by_price_band(y_test.tolist(), preds,500000,650000))

MAE 417662.81495664734
MPE 0.26728545984437707
MAE Band 228853.4664968705


## LASSO

In [42]:
lasso = Lasso().fit(x_train,y_train)
preds = lasso.predict(x_train)
print('MAE',mean_absolute_error(y_train,preds))
print('MPE',mean_percent_error(y_train.tolist(), preds))
print('MAE Band',mae_by_price_band(y_train.tolist(), preds,650000,750000))

C:\Users\Will\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.594040400029999e+16, tolerance: 18907816810856.277
  model = cd_fast.enet_coordinate_descent(


MAE 328445.4682929207
MPE 0.3016004899058156
MAE Band 227058.9395060901


In [43]:
preds = lasso.predict(x_test)
print('MAE',mean_absolute_error(y_test, preds))
print('MPE',mean_percent_error(y_test.tolist(), preds))
print('MAE Band',mae_by_price_band(y_test.tolist(), preds,500000,650000))

MAE 417838.2085120501
MPE 0.24601446236775817
MAE Band 227014.9273465015


## ELASTICNET

In [44]:
elasticNet = ElasticNet().fit(x_train,y_train)
preds = elasticNet.predict(x_train)
print('MAE',mean_absolute_error(y_train,preds))
print('MPE',mean_percent_error(y_train.tolist(), preds))
print('MAE Band',mae_by_price_band(y_train.tolist(), preds,650000,750000))

C:\Users\Will\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.534116924414139e+16, tolerance: 18907816810856.277
  model = cd_fast.enet_coordinate_descent(


MAE 413491.8269115377
MPE 0.6883323865314053
MAE Band 119172.42762715295


In [45]:
preds = elasticNet.predict(x_test)
print('MAE',mean_absolute_error(y_test, preds))
print('MPE',mean_percent_error(y_test.tolist(), preds))
print('MAE Band',mae_by_price_band(y_test.tolist(), preds,500000,650000))

MAE 516605.91179693316
MPE 0.6201658125135874
MAE Band 142176.88832492073


## RANDOM FOREST

In [46]:
from sklearn.ensemble import RandomForestRegressor

In [47]:
regr = RandomForestRegressor(max_depth=120, random_state=0)
regr.fit(x_train,y_train)
preds = regr.predict(x_train)
print('MAE',mean_absolute_error(y_train,preds))
print('MPE',mean_percent_error(y_train.tolist(), preds))
print('MAE Band',mae_by_price_band(y_train.tolist(), preds,650000,750000))

MAE 136406.9324395332
MPE 0.10256350071155591
MAE Band 89059.10816319947


In [48]:
preds = regr.predict(x_test)
print('MAE',mean_absolute_error(y_test, preds))
print('MPE',mean_percent_error(y_test.tolist(), preds))
print('MAE Band',mae_by_price_band(y_test.tolist(), preds,500000,650000))

MAE 334725.57054062444
MPE 0.1334350415287744
MAE Band 149079.96288052964


#### RANDOM FOREST DIFFERENT DEPTHS

In [67]:
for dep in [10, 20, 30,40, 50,60,70,80,90,100]:
    print(dep)
    regr = RandomForestRegressor(max_depth=dep, n_estimators = 100, random_state=0)
    regr.fit(scikit_dr_df.drop(columns =['SALE_PRICE'],axis = 1),scikit_dr_df['SALE_PRICE'])
    preds = regr.predict(scikit_dr_df.drop(columns =['SALE_PRICE'],axis = 1))
    print('MAE',mean_absolute_error(scikit_dr_df['SALE_PRICE'],preds))
    print('MPE',mean_percent_error(scikit_dr_df['SALE_PRICE'].tolist(), preds))
    print('MAE Band',mae_by_price_band(scikit_dr_df['SALE_PRICE'].tolist(), preds,650000,750000))
    preds = regr.predict(scikit_dr_Y_df.drop(columns =['SALE_PRICE'],axis = 1))
    print('MAE',mean_absolute_error(scikit_dr_Y_df['SALE_PRICE'], preds))
    print('MPE',mean_percent_error(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds))
    print('MAE Band',mae_by_price_band(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds,500000,650000))
    print()
    print()
    print()

10
MAE 290093.4953064403
MPE 0.32436580923500447
MAE Band 182139.8888184122
MAE 381173.7594272299
MPE 0.28138396071411725
MAE Band 177846.48852470913



20
MAE 216011.20556294604
MPE 0.21177479665472249
MAE Band 143845.54610955433
MAE 346802.2007590361
MPE 0.20109033221175723
MAE Band 161967.47377281217



30
MAE 171640.1656424412
MPE 0.14917423746610756
MAE Band 118712.79522612432
MAE 336883.19645714946
MPE 0.1597823721887532
MAE Band 151238.80038199225



40
MAE 151077.54864573997
MPE 0.12114966909594754
MAE Band 105015.82743005689
MAE 334562.7979218734
MPE 0.1424105299938095
MAE Band 148837.8914594417



50
MAE 142489.4389629632
MPE 0.1098163950600223
MAE Band 96787.38451978039
MAE 334000.9845805737
MPE 0.1364071155602082
MAE Band 148700.05935615193



60
MAE 138812.0773817725
MPE 0.10524922703762324
MAE Band 92416.00254155413
MAE 335042.5249681956
MPE 0.13493060250944125
MAE Band 149484.59792126596



70
MAE 137219.92741273166
MPE 0.10335837828522153
MAE Band 90222.59288853445
MAE 

#### RANDOM FORESTS DIFFERENT NUMBER OF TREES

In [66]:
for ne in [10, 20, 30,40, 50,60,70,80,90,100]:
    print(ne)
    regr = RandomForestRegressor(max_depth=50, n_estimators = ne, random_state=0)
    regr.fit(scikit_dr_df.drop(columns =['SALE_PRICE'],axis = 1),scikit_dr_df['SALE_PRICE'])
    preds = regr.predict(scikit_dr_df.drop(columns =['SALE_PRICE'],axis = 1))
    print('MAE',mean_absolute_error(scikit_dr_df['SALE_PRICE'],preds))
    print('MPE',mean_percent_error(scikit_dr_df['SALE_PRICE'].tolist(), preds))
    print('MAE Band',mae_by_price_band(scikit_dr_df['SALE_PRICE'].tolist(), preds,650000,750000))
    preds = regr.predict(scikit_dr_Y_df.drop(columns =['SALE_PRICE'],axis = 1))
    print('MAE',mean_absolute_error(scikit_dr_Y_df['SALE_PRICE'], preds))
    print('MPE',mean_percent_error(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds))
    print('MAE Band',mae_by_price_band(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds,500000,650000))
    print()
    print()
    print()

10
MAE 147755.62808499264
MPE 0.11024000673541587
MAE Band 101925.5384430451
MAE 340219.45958465623
MPE 0.13731039856677335
MAE Band 153665.39971114878



20
MAE 145321.68637536978
MPE 0.11030451068085043
MAE Band 99495.61874653627
MAE 336673.78728539706
MPE 0.1354879810805253
MAE Band 148745.96311509676



30
MAE 144004.5671275542
MPE 0.11005732899726169
MAE Band 98473.59098361313
MAE 334455.6944725035
MPE 0.13627424111413963
MAE Band 149133.75139898076



40
MAE 143631.64571779795
MPE 0.11017815168796842
MAE Band 97950.4689752032
MAE 334722.81345432095
MPE 0.13505813010381243
MAE Band 148321.29133976958



50
MAE 143269.21726899323
MPE 0.11011829132266933
MAE Band 97610.4309093056
MAE 333131.49258974945
MPE 0.13482053772055866
MAE Band 148211.33760204096



60
MAE 142983.4366604736
MPE 0.10988010660293095
MAE Band 97319.29373155256
MAE 333091.8489622282
MPE 0.13483449177444343
MAE Band 147256.17996439428



70
MAE 142740.6638148968
MPE 0.10964933288621878
MAE Band 97012.8355399837
MA

## GRADIENT BOOSTING

In [49]:
from sklearn.ensemble import GradientBoostingRegressor

In [50]:
regGB = GradientBoostingRegressor(max_depth=50, random_state=0)
regGB.fit(x_train,y_train)
preds = regGB.predict(x_train)
print('MAE',mean_absolute_error(y_train,preds))
print('MPE',mean_percent_error(y_train.tolist(), preds))
#     print('MAE Band',mae_by_price_band(scikit_dr_df['SALE_PRICE'].tolist(), preds,650000,750000))
preds = regGB.predict(x_test)
print('MAE',mean_absolute_error(y_test, preds))
print('MPE',mean_percent_error(y_test.tolist(), preds))
#     print('MAE Band',mae_by_price_band(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds,500000,650000))

MAE 86745.00534483024
MPE 0.06210416403143168
MAE 368797.07251463394
MPE 0.17052286379269221


#### GRADIENT BOOSTING DIFFERENT TREE DEPTHS

In [252]:
for dep in [10,20,30,40,50,60,70,80,90,100]:
    print()
    print()
    print(dep)
    regGB = GradientBoostingRegressor(max_depth=dep, random_state=0)
    regGB.fit(x_train,y_train)
    preds = regGB.predict(x_train)
    print('MAE',mean_absolute_error(y_train,preds))
    print('MPE',mean_percent_error(y_train.tolist(), preds))
#     print('MAE Band',mae_by_price_band(scikit_dr_df['SALE_PRICE'].tolist(), preds,650000,750000))
    preds = regGB.predict(x_test)
    print('MAE',mean_absolute_error(y_test, preds))
    print('MPE',mean_percent_error(y_test.tolist(), preds))
#     print('MAE Band',mae_by_price_band(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds,500000,650000))



10
MAE 239101.05075725474
MPE 0.2608417032026563
MAE 350849.9812043033
MPE 0.2659510908410551


20
MAE 149043.4463058618
MPE 0.1390589246064452
MAE 336541.6392653646
MPE 0.2137939677316085


30
MAE 102663.48061969085
MPE 0.07977115070725481
MAE 355329.77361510874
MPE 0.20144588456700177


40
MAE 88527.3638831938
MPE 0.06386751027856677
MAE 364819.63215947605
MPE 0.17893480198182907


50
MAE 86745.00534483024
MPE 0.06210416403143168
MAE 368797.07251463394
MPE 0.17052286379269221


60
MAE 86597.35976208285
MPE 0.06197515352176548
MAE 371494.7727510928
MPE 0.1684699748165555


70
MAE 86587.40892661198
MPE 0.061964669882885154
MAE 373693.6233730117
MPE 0.16708182372230204


80
MAE 86586.03187945628
MPE 0.06196325743081171
MAE 373364.5153425674
MPE 0.16860007189189508


90
MAE 86585.90225544975
MPE 0.06196313720457571
MAE 374046.9134240711
MPE 0.16794411573969426


100
MAE 86585.89303309488
MPE 0.061963132458158016
MAE 373990.51214191085
MPE 0.1681323486603491


## Different Time Periods

#### This shows the change in accuracy based on the month tested.  Our original test period happened to fit the training data extremely closely so we were gettting MPE of about 5%.  This was the test we ran that showed us that wasn't generalizable to the whole dataset.

In [137]:
scikit_df = all_sales_df.copy()
one_hots = ['BOROUGH','BUILDING CLASS CATEGORY','ZIP CODE','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE']
scikit_df = pd.get_dummies(scikit_df, columns = one_hots, prefix = one_hots)
scikit_df.drop('BLOCK',inplace = True, axis = 1)
scikit_df.set_index('SALE DATE', inplace=True)

test_dates = [['2005-01-01' , '2007-12-31','2008-01-01' , '2008-01-31'],
              ['2006-01-01' , '2008-12-31','2009-01-01' , '2009-01-31'],
              ['2007-01-01' , '2009-12-31','2010-01-01' , '2010-01-31'],
              ['2008-01-01' , '2010-12-31','2011-01-01' , '2011-01-31'],
              ['2009-01-01' , '2011-12-31','2012-01-01' , '2012-01-31'],
              ['2010-01-01' , '2012-12-31','2013-01-01' , '2013-01-31'],
              ['2011-01-01' , '2013-12-31','2014-01-01' , '2014-01-31'],
              ['2012-01-01' , '2014-12-31','2015-01-01' , '2015-01-31'],
              ['2013-01-01' , '2015-12-31','2016-01-01' , '2016-01-31'],
              ['2014-01-01' , '2016-12-31','2017-01-01' , '2017-01-31'],
              ['2015-01-01' , '2017-12-31','2018-01-01' , '2018-01-31'],
              ['2016-01-01' , '2018-12-31','2019-01-01' , '2019-01-31'],
             
             
             ]

for dates in test_dates:
# date_range_sample = ['2005-01-01' , '2007-12-31']
# date_range_target = ['2008-01-01' , '2008-01-31']

    #Remove any columns in the data range that don't have any hits on the one hot
    temp_df = scikit_df.loc[dates[0] : dates[1]]
    print(temp_df.shape)
    scikit_dr_df = temp_df.loc[:, (temp_df != 0).any(axis=0)]
    print(scikit_dr_df.shape)

    #Take the same columns as the sample date range so target date range can be passed to scikit
    temp_target_df = scikit_df.loc[dates[2] : dates[3]]
    scikit_dr_Y_df = temp_target_df[scikit_dr_df.columns]

    scikit_dr_df = Convert_dates(scikit_dr_df)
    scikit_dr_df


    for col in scikit_dr_df:
        if col.startswith('Date_'):
            scikit_dr_Y_df[col] = 0
    # scikit_dr_Y_df = Convert_dates(scikit_dr_Y_df)
    scikit_dr_Y_df



    x_train = scikit_dr_df.drop(columns =['SALE_PRICE'],axis = 1)
    y_train = scikit_dr_df['SALE_PRICE']

    x_test = scikit_dr_Y_df.drop(columns =['SALE_PRICE'],axis = 1)
    y_test = scikit_dr_Y_df['SALE_PRICE']



    regr = RandomForestRegressor(max_depth=120, random_state=0)
    regr.fit(x_train,y_train)
    preds = regr.predict(x_train)
    print(dates[2])
    print('MAE',mean_absolute_error(y_train,preds))
    print('MPE',mean_percent_error(y_train.tolist(), preds))
    preds = regr.predict(x_test)
    print('Test')
    print('MAE',mean_absolute_error(y_test, preds))
    print('MPE',mean_percent_error(y_test.tolist(), preds))
    print()
    print()
    print()

(229476, 573)
(229476, 394)


<ipython-input-60-caac207595d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df[new_col_name] = 0
<ipython-input-137-a6126a3756fa>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scikit_dr_Y_df[col] = 0


2008-01-01
MAE 136406.9324395332
MPE 0.10256350071155591
Test
MAE 334725.57054062444
MPE 0.1334350415287744



(202275, 573)
(202275, 402)


<ipython-input-60-caac207595d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df[new_col_name] = 0
<ipython-input-137-a6126a3756fa>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scikit_dr_Y_df[col] = 0


2009-01-01
MAE 153981.3443918266
MPE 0.10785934838251017
Test
MAE 270328.76457747887
MPE 0.2437350384395053



(169278, 573)
(169278, 397)


...

2017-01-01
MAE 172100.81211382363
MPE 0.11152766575007654
Test
MAE 263824.0676456533
MPE 0.016613741879062914



(115516, 573)
(115516, 473)


<ipython-input-60-caac207595d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df[new_col_name] = 0
<ipython-input-137-a6126a3756fa>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scikit_dr_Y_df[col] = 0


2018-01-01
MAE 146505.67807491616
MPE 0.09717866436375563
Test
MAE 393593.8602466328
MPE 0.2719858048300379



(112702, 573)
(112702, 430)


<ipython-input-60-caac207595d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df[new_col_name] = 0
<ipython-input-137-a6126a3756fa>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scikit_dr_Y_df[col] = 0


2019-01-01
MAE 112714.48289672432
MPE 0.07736203235552971
Test
MAE 308280.8652108634
MPE 0.1843702011206171





## TRAINING AND TESTING OVER ENTIRE DATE RANGE

#### DATE PREP

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
scikit_df = all_sales_df.copy()
# one_hots = ['BOROUGH','NEIGHBORHOOD','BUILDING CLASS CATEGORY','ZIP CODE','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE']
one_hots = ['BOROUGH','BUILDING CLASS CATEGORY','ZIP CODE','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE']
scikit_df = pd.get_dummies(scikit_df, columns = one_hots, prefix = one_hots)
scikit_df.drop('BLOCK',inplace = True, axis = 1)
scikit_df.set_index('SALE DATE', inplace=True)
# date_range_sample = ['2003-01-01' , '2007-12-31']
# date_range_target = ['2008-01-01' , '2008-01-31']

scikit_df = Convert_dates(scikit_df)

train, test = train_test_split(scikit_df, test_size=0.1)

# for col in scikit_dr_df:
#     if col.startswith('Date_'):
#         scikit_dr_Y_df[col] = 0
# scikit_dr_Y_df = Convert_dates(scikit_dr_Y_df)
# scikit_dr_Y_df

In [205]:
reg_all = LinearRegression().fit(train.drop(columns =['SALE_PRICE'],axis = 1),train['SALE_PRICE'])
preds = reg_all.predict(test.drop(columns =['SALE_PRICE'],axis = 1))
print('MAE',mean_absolute_error(test['SALE_PRICE'], preds))
print('MPE',mean_percent_error(test['SALE_PRICE'].tolist(), preds))
print('MAE Band',mae_by_price_band(test['SALE_PRICE'].tolist(), preds,500000,650000))

MAE 404618.6009498198
MPE 0.37511521801295966
MAE Band 258670.48997072878


In [206]:
ridge_all = Ridge().fit(train.drop(columns =['SALE_PRICE'],axis = 1),train['SALE_PRICE'])
preds = ridge_all.predict(test.drop(columns =['SALE_PRICE'],axis = 1))
print('MAE',mean_absolute_error(test['SALE_PRICE'], preds))
print('MPE',mean_percent_error(test['SALE_PRICE'].tolist(), preds))
print('MAE Band',mae_by_price_band(test['SALE_PRICE'].tolist(), preds,500000,650000))

C:\Users\Will\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=2.25089e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


MAE 404463.7200146408
MPE 0.37517200445771237
MAE Band 258621.94859338985


In [207]:
lasso_all = Lasso().fit(train.drop(columns =['SALE_PRICE'],axis = 1),train['SALE_PRICE'])
preds = lasso_all.predict(test.drop(columns =['SALE_PRICE'],axis = 1))
print('MAE',mean_absolute_error(test['SALE_PRICE'], preds))
print('MPE',mean_percent_error(test['SALE_PRICE'].tolist(), preds))
print('MAE Band',mae_by_price_band(test['SALE_PRICE'].tolist(), preds,500000,650000))

C:\Users\Will\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.645217313102164e+17, tolerance: 89646158559851.02
  model = cd_fast.enet_coordinate_descent(


MAE 404441.62495287025
MPE 0.3752178501068814
MAE Band 258563.09780427977


In [208]:
elasticNet_all = ElasticNet().fit(train.drop(columns =['SALE_PRICE'],axis = 1),train['SALE_PRICE'])
preds = elasticNet_all.predict(test.drop(columns =['SALE_PRICE'],axis = 1))
print('MAE',mean_absolute_error(test['SALE_PRICE'], preds))
print('MPE',mean_percent_error(test['SALE_PRICE'].tolist(), preds))
print('MAE Band',mae_by_price_band(test['SALE_PRICE'].tolist(), preds,500000,650000))

C:\Users\Will\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.1442874197287174e+17, tolerance: 89646158559851.02
  model = cd_fast.enet_coordinate_descent(


MAE 511589.26233962516
MPE 0.8490143971186013
MAE Band 187705.39586078664


In [ ]:
regrf = RandomForestRegressor(max_depth=50,n_estimators=100, random_state=0)
regrf.fit(train.drop(columns =['SALE_PRICE'],axis = 1),train['SALE_PRICE'])
preds = regrf.predict(train.drop(columns =['SALE_PRICE'],axis = 1))
print('MAE',mean_absolute_error(test['SALE_PRICE'], preds))
print('MPE',mean_percent_error(test['SALE_PRICE'].tolist(), preds))
print('MAE Band',mae_by_price_band(test['SALE_PRICE'].tolist(), preds,500000,650000))

In [209]:
regGB_all = GradientBoostingRegressor(max_depth=80, random_state=0).fit(train.drop(columns =['SALE_PRICE'],axis = 1),train['SALE_PRICE'])
preds = regGB_all.predict(test.drop(columns =['SALE_PRICE'],axis = 1))
print('MAE',mean_absolute_error(test['SALE_PRICE'], preds))
print('MPE',mean_percent_error(test['SALE_PRICE'].tolist(), preds))
print('MAE Band',mae_by_price_band(test['SALE_PRICE'].tolist(), preds,500000,650000))

MAE 322709.01287193195
MPE 0.19548375241767027
MAE Band 178738.50440925208


#### Testing time for random forest based on number of cores(n_jobs)

In [ ]:
for jobs in [1,5,10]:
    print(jobs)
    start_time= time.time()
    regr = RandomForestRegressor(max_depth=30,n_estimators=100, n_jobs = jobs, random_state=0)
    regr.fit(train.drop(columns =['SALE_PRICE'],axis = 1),train['SALE_PRICE'])
    preds = regr.predict(train.drop(columns =['SALE_PRICE'],axis = 1))
    print(time.time()-start_time)
    print()

## Polynomial Features

#### We experimented with polynomial feature transformations but becasue of how many features there were after one hot encoding we believe that the poor performance may be due to the curse of dimensionality.

In [210]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2, interaction_only=)
transformed_features = PolyFeat(scikit_dr_df.drop(columns =['SALE_PRICE'],axis = 1), one_hots)
reg = LinearRegression().fit(transformed_features,scikit_dr_df['SALE_PRICE'])
preds = reg.predict( PolyFeat(scikit_dr_Y_df.drop(columns =['SALE_PRICE'],axis = 1),one_hots))
print('MAE: ' , print(mean_absolute_error(scikit_dr_Y_df['SALE_PRICE'], preds)))
print('MPE',mean_percent_error(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds))
print('MAE BAND',mae_by_price_band(scikit_dr_Y_df['SALE_PRICE'].tolist(), preds,300000,500000))

12899282557122.031
MAE:  None
MPE 10896693.294032576
MAE BAND 2188365688196.2502


In [ ]:
transformed_features.shape

## KNN EXPERIMENTS

#### The accuracy here wasn't very good and it wouldn't be feasible for distributed training.  It's possible that with enough data and a way to scale prices based on overall market trends this would be more accurate but it would still wouldn't be parallelizable.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(x_train,y_train)

In [ ]:
preds = neigh.predict(x_test)
print('MAE',mean_absolute_error(y_test, preds))
print('MPE',mean_percent_error(y_test.tolist(), preds))
print('MAE Band',mae_by_price_band(y_test.tolist(), preds,500000,650000))
print(mean_absolute_error(y_test, preds))
print(mean_percent_error(y_test.tolist(), preds))